In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator



from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow_addons as tfa

In [2]:
TRAIN_DIR = '../../../data/cell_type/training'
TEST_DIR = '../../../data/cell_type/testing'
VAL_DIR = '../../../data/cell_type/validation'
INPUT_WIDTH = 32
INPUT_HEIGHT = 32
IMG_SIZE = (32,32)

training_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(INPUT_WIDTH,INPUT_HEIGHT),
    class_mode='categorical',
    batch_size=64
)

validation_generator = validation_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(INPUT_WIDTH,INPUT_HEIGHT),
    class_mode='categorical',
    batch_size=64
)

test_dataset = image_dataset_from_directory(TEST_DIR,
                                                  shuffle=True,
                                                  batch_size=64,
                                                  image_size=(32,32),
                                                  interpolation='nearest',
                                                  label_mode='categorical'
                                            )

Found 7915 images belonging to 4 classes.
Found 990 images belonging to 4 classes.
Found 991 files belonging to 4 classes.


In [3]:
from tensorflow.keras.applications.resnet_v2 import ResNet101V2

resnet_101V2_model = ResNet101V2(input_shape = (INPUT_WIDTH, INPUT_HEIGHT, 3), 
                                include_top = False, 
                                weights = None)

In [4]:
for layer in resnet_101V2_model.layers:
    layer.trainable = True

In [5]:
resnet_101V2_model.summary()

Model: "resnet101v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 18, 18, 64)   0           conv1_conv[0][0]                 
________________________________________________________________________________________

In [6]:
last_layer = resnet_101V2_model.get_layer('post_relu')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 1, 1, 2048)


In [7]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(4)

# x = base_model(x, training=True)
x = global_average_layer(last_output)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(resnet_101V2_model.input, outputs)

In [8]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))



In [9]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy', f1_score ,precision_m, recall_m])

/home/trungngo/Desktop/venv/colon-cancer-classification/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [10]:
len(model.trainable_variables)

344

In [11]:
#Early stopping when training_acc > 0.99
class myCallBack(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') >= 0.99:
            print('\nReached 99% accuracy so cancelling training!')
            self.model.stop_training = True

history = model.fit(train_generator, 
                    epochs=150, 
                    #steps_per_epoch=(total_training_imgs // 128), 
                    validation_data = validation_generator, 
                    verbose = 1,
                    callbacks=[myCallBack()]
                    #validation_steps=(total_val_imgs // 64)
)

Epoch 1/150
124/124 [==============================] - 14s 50ms/step - loss: 4.2417 - accuracy: 0.1858 - f1_score: 0.2200 - precision_m: 0.2143 - recall_m: 0.2276 - val_loss: 1.3720 - val_accuracy: 0.1404 - val_f1_score: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/150
124/124 [==============================] - 5s 40ms/step - loss: 2.3498 - accuracy: 0.1785 - f1_score: 0.2452 - precision_m: 0.2408 - recall_m: 0.2507 - val_loss: 1.3463 - val_accuracy: 0.1424 - val_f1_score: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 3/150
124/124 [==============================] - 5s 37ms/step - loss: 1.9514 - accuracy: 0.1658 - f1_score: 0.2321 - precision_m: 0.2403 - recall_m: 0.2261 - val_loss: 1.8308 - val_accuracy: 0.1424 - val_f1_score: 0.0037 - val_precision_m: 0.0312 - val_recall_m: 0.0020
Epoch 4/150
124/124 [==============================] - 5s 39ms/step - loss: 1.8844 - accuracy: 0.1674 - f1_score: 0.1844 - precision_m: 0.2294 - re

KeyboardInterrupt: 